# Imports

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import os

# API Information

In [ ]:
# API key (expires after 24 hours)
api_key = "RGAPI-51c2aa98-8c71-42ee-bb0b-5397eb8e786d"

# Summoner Information

In [ ]:
# List of Summoner Names and Tags 
summoner_name_list = ["KRP%20Kirai/KRP", "KRP%20SummerChild/SUCH", "KRP%20Nightreign/KRP"]
#summoner_name_list = ["KRP%20SummerChild/SUCH"]

In [ ]:
summoner_name_url = "https://europe.api.riotgames.com/riot/account/v1/accounts/by-riot-id/"

In [ ]:
summoner_data = []

In [ ]:
# Loop through each summoner name and fetch data
for summoner_name in summoner_name_list:
    full_summoner_name_url = summoner_name_url + summoner_name + "?api_key=" + api_key
    
    # Send request
    resp = requests.get(full_summoner_name_url)
    
    # Check if the request was successful
    if resp.status_code == 200:
        player_info = resp.json()
        summoner_data.append(player_info)  # Append the JSON response to the list
    else:
        print(f"Failed to fetch data for {summoner_name}: {resp.status_code}")

# Convert the list of dictionaries into a DataFrame
summoner_df = pd.DataFrame(summoner_data)

# Display the DataFrame
print(summoner_df)

In [ ]:
summoner_df.to_excel("summoner_df.xlsx", index=False)

print("Excel file saved: summoner_df.xlsx")

# Summoner Matches

### Unix Dates (Start-End Dates)

In [ ]:
# Define start and end dates
start_date = datetime(2021, 6, 16)
end_date = datetime.today()

# Generate date ranges
date_ranges = []
while start_date < end_date:
    week_end = start_date + timedelta(days=6)  # 7-day range
    week_end = week_end.replace(hour=23, minute=59, second=59)  # Ensure full day is covered
    
    unix_start = int(time.mktime(start_date.timetuple()))
    unix_end = int(time.mktime(week_end.timetuple()))  # Now includes full 7 days

    date_ranges.append([start_date.strftime('%Y-%m-%d'), week_end.strftime('%Y-%m-%d'), unix_start, unix_end])
    
    start_date += timedelta(days=7)  # Move to the next week


unix_dates_df = pd.DataFrame(date_ranges, columns=['Start Date', 'End Date', 'Unix Start', 'Unix End'])


print(unix_dates_df)

In [ ]:
start_str = start_date.strftime("%Y_%m_%d")
end_str = end_date.strftime("%Y_%m_%d")

## Data Source

We are using an Excel file that contains a list of summoner name-tags along with their corresponding Match IDs.  
For each summoner, the dataset includes their **first 100 matches** retrieved from the Riot API.

In [ ]:
# Create a list to store all match data
match_data_list = []

# Loop through each summoner
for index_summoner, summoner_row in summoner_df.iterrows():
    match_puuid = summoner_row['puuid']
    
    # Loop through each unix date range
    for index_date, date_row in unix_dates_df.iterrows():
        match_startTime = str(date_row['Unix Start'])
        match_endTime = str(date_row['Unix End'])
        
        match_url = (
            "https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/" +
            match_puuid +
            "/ids?startTime=" + match_startTime +
            "&endTime=" + match_endTime +
            "&start=0&count=100" +
            "&api_key=" + api_key
        )
        
        try:
            match_resp = requests.get(match_url)
            match_resp.raise_for_status()  # Raise error for bad responses
            matches = match_resp.json()  # This is the list of match IDs
        except requests.exceptions.RequestException as e:
            print(f"Error fetching matches for {summoner_row['gameName']} between {match_startTime} and {match_endTime}: {e}")
            matches = None
        
        # If matches found, add them to the list
        if matches:
            for match in matches:
                match_data_list.append({
                    "gameName": summoner_row['gameName'],
                    "tagLine": summoner_row['tagLine'],
                    "puuid": match_puuid,
                    "match_url": match_url,
                    "match_id": match,  # Store individual match ID
                    "startTime": match_startTime,
                    "endTime": match_endTime,
                    
                })

# Convert to DataFrame
match_data_list = pd.DataFrame(match_data_list)

# Display the DataFrame
print(match_data_list)

file_path = (
    r"match_data_list_" +
    str(unix_dates_df['Unix Start'].min()) + "_" +
    str(unix_dates_df['Unix End'].max()) + ".xlsx"
)
match_data_list.to_excel(file_path, index=False)

print("Excel file named match_data_list.xlsx saved successfully!")

### URLs for Match IDs

In [ ]:
test_match_id_list = []
matchID_url_base = "https://europe.api.riotgames.com/lol/match/v5/matches/" 

for match_id in match_data_list['match_id']:
     matchID_url =  matchID_url_base + match_id + "?api_key=" + api_key
     
     test_match_id_list.append({
                    "match_id": match_id,
                    "matchID_url": matchID_url    
                })
       
    
#print(test_match_id_list)
# Convert the list to a DataFrame
test_match_id_list_df = pd.DataFrame(test_match_id_list)

file_path = (
    r"test_match_id_list_" +
    str(unix_dates_df['Unix Start'].min()) + "_" +
    str(unix_dates_df['Unix End'].max()) + ".xlsx"
)
test_match_id_list_df.to_excel(file_path, index=False)


print("Excel file named test_match_id_list saved successfully!")

### Riot Match Participants Data

In [ ]:
# Get list of URLs from the DataFrame column
riot_links = test_match_id_list_df["matchID_url"].tolist()

# Store player-level data
all_players_data = []

for url in riot_links:
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        match_id = data['metadata']['matchId']
        game_duration = data['info']['gameDuration']
        game_mode = data['info']['gameMode']
        participants_info = data['info']['participants']
        participants_puuids = data['metadata']['participants']

        # Optional check to skip malformed matches
        if len(participants_info) != len(participants_puuids):
            print(f"Skipping match {match_id} due to participant length mismatch")
            continue

        for i, player in enumerate(participants_info):
            puuid = participants_puuids[i]  # safely assigned
            player_data = {
                "Match ID": match_id,
                "Game Mode": game_mode,
                "Game Duration": game_duration,
                "PUUID": puuid,
                "Summoner Name": player.get("summonerName"),
                "Champion Name": player.get("championName"),
                "Team Position": player.get("teamPosition"),
                "Lane": player.get("lane"),
                "Kills": player.get("kills"),
                "Deaths": player.get("deaths"),
                "Assists": player.get("assists"),
                "Win": player.get("win"),
                "Total Damage To Champs": player.get("totalDamageDealtToChampions"),
                "Total Damage Taken": player.get("totalDamageTaken"),
                "Gold Earned": player.get("goldEarned"),
                "Total Minions Killed": player.get("totalMinionsKilled"),
                "Vision Score": player.get("visionScore"),
            }

            all_players_data.append(player_data)
        time.sleep(2.0)  # Respect Riot API rate limits

    except requests.exceptions.RequestException as e:
        print(f" Error fetching {url}: {e}")

# Convert to DataFrame
df = pd.DataFrame(all_players_data)

# Save to Excel with dynamic filename
file_path = (
    r"riot_match_participants_" +
    str(unix_dates_df['Unix Start'].min()) + "_" +
    str(unix_dates_df['Unix End'].max()) + ".xlsx"
)
df.to_excel(file_path, index=False)

print("Participants data successfully saved")


### Data Source for Champion Matchup

In [ ]:
# Sort dataframe by Match ID and PUUID to ensure consistent player ordering
df = df.sort_values(['Match ID', 'PUUID'])

# Assign a player number (1-10) within each match for team separation
df['Player_Number'] = df.groupby('Match ID').cumcount() + 1  


# Filter the dataset to only include rows for the target player
player_df = df[df['PUUID'] == target_puuid].copy()

def get_other_champions_conditional(row, full_df):

    match_id = row['Match ID']
    player_num = row['Player_Number']

    # Team 1 consists of players 1-5, Team 2 consists of players 6-10
    if player_num <= 5:
        # Player is on Team 1, so opposing champions are from Team 2 (players 6-10)
        other_champs = full_df[(full_df['Match ID'] == match_id) & 
                               (full_df['Player_Number'] > 5)]['Champion Name'].unique()
    else:
        # Player is on Team 2, so opposing champions are from Team 1 (players 1-5)
        other_champs = full_df[(full_df['Match ID'] == match_id) & 
                               (full_df['Player_Number'] <= 5)]['Champion Name'].unique()

    return list(other_champs)

# Apply the function to each row for the target player to get the opposing champions list
player_df['Other Champions List'] = player_df.apply(lambda row: get_other_champions_conditional(row, df), axis=1)

# Explode the list of opposing champions to have one champion per row for easier analysis
result = player_df.explode('Other Champions List')

# Select relevant columns and rename for clarity
result = result[['Match ID', 'PUUID', 'Champion Name', 'Other Champions List', 'Win']]\
         .rename(columns={'Other Champions List': 'Other Champion'})

print(result)

# Define output file path dynamically based on date range (assumes unix_dates_df is defined)
file_path = (
    r"heatmap_" +
    str(unix_dates_df['Unix Start'].min()) + "_" +
    str(unix_dates_df['Unix End'].max()) + ".xlsx"
)

# Export the processed data to Excel for use in dashboard visualizations
result.to_excel(file_path, index=False)

print("Heatmap data successfully saved")
